In [1]:
import tensorflow as tf
import numpy as np
import nengo_dl
import nengo
import random

#tf.reset_default_graph()
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


# TF Model definition

In [4]:
# input
inp = tf.keras.Input(shape=(28, 28, 1))

# convolutional layers
conv0 = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation=tf.nn.relu,
)(inp)

# Default pool_size = (2,2), padding = "valid", data_format = "channels_last".
max_pool0 = tf.keras.layers.MaxPool2D()(conv0) 

conv1 = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    activation=tf.nn.relu,
)(max_pool0)

# max_pool1 = tf.keras.layers.MaxPool2D()(conv1) 

# conv2 = tf.keras.layers.Conv2D(
#     filters=64,
#     kernel_size=3,
#     strides=2,
#     activation=tf.nn.relu,
# )(max_pool1)


# fully connected layer
flatten = tf.keras.layers.Flatten()(conv1)
dense = tf.keras.layers.Dense(units=10, activation="softmax")(flatten)

model = tf.keras.Model(inputs=inp, outputs=dense)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                23050     
Total params: 41,866
Trainable params: 41,866
Non-trainable params: 0
_________________________________________________________

## TF Model Compilation and Evaluation

In [6]:
model.compile(
  optimizer=tf.optimizers.Adam(0.001),
  loss=tf.losses.SparseCategoricalCrossentropy(),
  metrics=[tf.metrics.sparse_categorical_accuracy])
model.fit(train_images, train_labels, epochs=4)

Epoch 1/4
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2810 - sparse_categorical_accuracy: 0.9438
Epoch 2/4
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0735 - sparse_categorical_accuracy: 0.9780
Epoch 3/4
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0582 - sparse_categorical_accuracy: 0.9829
Epoch 4/4
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0503 - sparse_categorical_accuracy: 0.9851


In [7]:
model.evaluate(test_images, test_labels)
# [0.06594502180814743, 0.9804999828338623]

313/313 [==============================] - 1s 5ms/step - loss: 0.0663 - sparse_categorical_accuracy: 0.9811


[0.06628934293985367, 0.9811000227928162]

# Check Nengo DL MaxPool

In [8]:
n_steps = 40
np.random.seed(100)
ndl_model_3 = nengo_dl.Converter(model, 
                               swap_activations={tf.nn.relu: nengo.SpikingRectifiedLinear()},
                               scale_firing_rates=5,
                               max_to_avg_pool=True,
                               synapse=0.005)

with ndl_model_3.net:
  nengo_dl.configure_settings(stateful=False)

/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:588: UserWarning: Activation type <function softmax at 0x2ab70f1b9c20> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation


In [9]:
ndl_test_images = np.tile(
  test_images.reshape((test_images.shape[0], 1, -1)), (1, n_steps, 1))
ndl_input_3 = ndl_model_3.inputs[inp]
ndl_output_3 = ndl_model_3.outputs[dense]

In [10]:
with nengo_dl.Simulator(
  ndl_model_3.net, minibatch_size=100) as sim:
  data3 = sim.predict({ndl_input_3: ndl_test_images[:200]})

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


In [11]:
acc = 0
for pred, true in zip(data3[ndl_output_3][:, -1, :], test_labels):
  if np.argmax(pred) == true:
    acc += 1
print(acc/200)

0.755


# Conversion from TF to spiking Nengo DL model with custom max to avg Node

In [12]:
n_steps = 40
np.random.seed(100)
ndl_model_4 = nengo_dl.Converter(model, 
                               swap_activations={tf.nn.relu: nengo.SpikingRectifiedLinear()},
                               scale_firing_rates=5,
                               synapse=0.005)

with ndl_model_4.net:
  nengo_dl.configure_settings(stateful=False)

/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")


## Check connections before replacing

In [13]:
ndl_model_4.net.all_nodes, ndl_model_4.net.all_ensembles, ndl_model_4.net.all_connections

([<Node "input_1" at 0x2aba991c7690>,
  <Node "conv2d.0.bias" at 0x2aba992efe90>,
  <Node "conv2d.0.bias_relay" at 0x2aba992efe10>,
  <TensorNode "max_pooling2d" at 0x2aba9917c5d0>,
  <Node "conv2d_1.0.bias" at 0x2aba9919cdd0>,
  <Node "conv2d_1.0.bias_relay" at 0x2aba9919cf10>,
  <TensorNode "dense.0" at 0x2aba991ae490>,
  <Node "dense.0.bias" at 0x2aba991a4e10>],
 [<Ensemble "conv2d.0" at 0x2aba94d21f50>,
  <Ensemble "conv2d_1.0" at 0x2aba94adda10>],
 [<Connection at 0x2aba992f7090 from <Node "conv2d.0.bias"> to <Node "conv2d.0.bias_relay">>,
  <Connection at 0x2aba991dbed0 from <Node "conv2d.0.bias_relay"> to <Neurons of <Ensemble "conv2d.0">>>,
  <Connection at 0x2aba94b4a110 from <Node "input_1"> to <Neurons of <Ensemble "conv2d.0">>>,
  <Connection at 0x2aba9915df90 from <Neurons of <Ensemble "conv2d.0">> to <TensorNode "max_pooling2d">>,
  <Connection at 0x2aba9919ce50 from <Node "conv2d_1.0.bias"> to <Node "conv2d_1.0.bias_relay">>,
  <Connection at 0x2aba94add410 from <Node "c

In [14]:
conn_from_conv0_to_max_node0 = ndl_model_4.net.all_connections[3]
print(conn_from_conv0_to_max_node0.pre_obj, conn_from_conv0_to_max_node0.post_obj)
conn_from_max_node0_to_conv1 = ndl_model_4.net.all_connections[6]
print(conn_from_max_node0_to_conv1.pre_obj, conn_from_max_node0_to_conv1.post_obj)

<Neurons of <Ensemble "conv2d.0">> <TensorNode "max_pooling2d">
<TensorNode "max_pooling2d"> <Neurons of <Ensemble "conv2d_1.0">>


In [15]:
# conn_from_conv1_to_max_node1 = ndl_model_3.net.all_connections[7]
# print(conn_from_conv1_to_max_node1.pre_obj, conn_from_conv1_to_max_node1.post_obj)
# conn_from_max_node1_to_conv2 = ndl_model_3.net.all_connections[10]
# print(conn_from_max_node1_to_conv2.pre_obj, conn_from_max_node1_to_conv2.post_obj)

## Replacing the Max Node with Custom Avg Node

In [16]:
def compute_max_to_average_conv0(t, inp):
  print(inp[np.where(inp)])
  inp = inp.reshape(26, 26, 32)
  ret = np.zeros((13, 13, 32))
  
#   for chnl in range(inp.shape[2]): 
#     for r in range(13):
#       for c in range(13):
#         ret[r, c, chnl] = np.mean(inp[r*2:r*2+2, c*2:c*2+2, chnl])
  
  return ret.flatten()
        
# def compute_max_to_average_conv1(t, inp):
#   inp = inp.reshape(6, 6, 64)
#   ret = np.zeros((3, 3, 64))
  
#   for chnl in range(inp.shape[2]):
#     for r in range(3):
#       for c in range(3):
#         ret[r, c, chnl] = np.mean(inp[r*2:r*2+2, c*2:c*2+2, chnl])
        
#   return ret.flatten()

In [17]:
with ndl_model_4.net:
  new_node0 = nengo.Node(compute_max_to_average_conv0, size_in=21632, label="Custom Node 0")  
  #new_node1 = nengo.Node(compute_max_to_average_conv1, size_in=2304, label="Custom Node 1")
  
  # Connection from Conv0 to custom new_node0
  conn_from_conv0_to_max_node0 = ndl_model_4.net.all_connections[3]
  nengo.Connection(
    conn_from_conv0_to_max_node0.pre_obj,
    new_node0,
    transform=conn_from_conv0_to_max_node0.transform,
    synapse=conn_from_conv0_to_max_node0.synapse,
    function=conn_from_conv0_to_max_node0.function
  )
  
  # Connection from new_node0 to Conv1
  conn_from_max_node0_to_conv1 = ndl_model_4.net.all_connections[6]
  nengo.Connection(
    new_node0,
    conn_from_max_node0_to_conv1.post_obj,
    transform=conn_from_max_node0_to_conv1.transform,
    synapse=conn_from_max_node0_to_conv1.synapse,
    function=conn_from_max_node0_to_conv1.function
  )
  
  #################################################
  
#   # Connection from Conv1 to custom new_node1
#   conn_from_conv1_to_max_node1 = ndl_model_3.net.all_connections[7]
#   nengo.Connection(
#     conn_from_conv1_to_max_node1.pre_obj,
#     new_node1,
#     transform=conn_from_conv1_to_max_node1.transform,
#     synapse=conn_from_conv1_to_max_node1.synapse,
#     function=conn_from_conv1_to_max_node1.function
#     )
  
#   # Connection from custom new_node1 to Conv2.
#   conn_from_max_node1_to_conv2 = ndl_model_3.net.all_connections[10]
#   nengo.Connection(
#     new_node1,
#     conn_from_max_node1_to_conv2.post_obj,
#     transform=conn_from_max_node1_to_conv2.transform,
#     synapse=conn_from_max_node1_to_conv2.synapse,
#     function=conn_from_max_node1_to_conv2.function
#   )
  
  # Delete old connections.
  ndl_model_4.net._connections.remove(conn_from_conv0_to_max_node0)
  ndl_model_4.net._connections.remove(conn_from_max_node0_to_conv1)
  
  #ndl_model_3.net._connections.remove(conn_from_conv1_to_max_node1)
  #ndl_model_3.net._connections.remove(conn_from_max_node1_to_conv2)
  
  # Delete the two TensorNodes.
  ndl_model_4.net._nodes.remove(conn_from_conv0_to_max_node0.post_obj)
  #ndl_model_3.net._nodes.remove(conn_from_conv1_to_max_node1.post_obj)

[]


# Check if connection was successful

In [18]:
ndl_model_4.net.all_connections, ndl_model_4.net._nodes

([<Connection at 0x2aba992f7090 from <Node "conv2d.0.bias"> to <Node "conv2d.0.bias_relay">>,
  <Connection at 0x2aba991dbed0 from <Node "conv2d.0.bias_relay"> to <Neurons of <Ensemble "conv2d.0">>>,
  <Connection at 0x2aba94b4a110 from <Node "input_1"> to <Neurons of <Ensemble "conv2d.0">>>,
  <Connection at 0x2aba9919ce50 from <Node "conv2d_1.0.bias"> to <Node "conv2d_1.0.bias_relay">>,
  <Connection at 0x2aba94add410 from <Node "conv2d_1.0.bias_relay"> to <Neurons of <Ensemble "conv2d_1.0">>>,
  <Connection at 0x2aba991a43d0 from <Node "dense.0.bias"> to <TensorNode "dense.0">>,
  <Connection at 0x2aba991a4e90 from <Neurons of <Ensemble "conv2d_1.0">> to <TensorNode "dense.0">>,
  <Connection at 0x2aba992f7b90 from <Neurons of <Ensemble "conv2d.0">> to <Node "Custom Node 0">>,
  <Connection at 0x2aba9919ce90 from <Node "Custom Node 0"> to <Neurons of <Ensemble "conv2d_1.0">>>],
 [<Node "input_1" at 0x2aba991c7690>,
  <Node "conv2d.0.bias" at 0x2aba992efe90>,
  <Node "conv2d.0.bias_r

In [19]:
ndl_test_images = np.tile(
  test_images.reshape((test_images.shape[0], 1, -1)), (1, n_steps, 1))
ndl_input_4 = ndl_model_4.inputs[inp]
ndl_output_4 = ndl_model_4.outputs[dense]

In [20]:
with nengo_dl.Simulator(
  ndl_model_4.net, minibatch_size=100) as sim:
  data4 = sim.predict({ndl_input_4: ndl_test_images[:200]})

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.         
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 

[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 20

[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 2

[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200.]

[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200.]

...

[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200.

[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 2

[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 

[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200.]
[200. 200. 200. 200

[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.]
[200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200. 200.
 200. 200. 200. 200

In [21]:
acc = 0
for pred, true in zip(data4[ndl_output_4][:, -1, :], test_labels):
  if np.argmax(pred) == true:
    acc += 1
print(acc/200)

0.14


In [22]:
k = np.zeros((10))
k[0] = 1
k[1] = 2
k[np.where(k)]

array([1., 2.])